In [ ]:
%pylab

In [ ]:
def sphere(x):
    return np.inner(x,x)

In [ ]:
class ArtificialBeeColonyAlgorithm(object):
    def __init__(self, 
                 objective_function, 
                 dimension, 
                 upper_bounds,
                 lower_bounds,
                 population_size, 
                 max_trial_limit, 
                 max_num_iterations,
                 debug=False):
        self._objective_function = objective_function
        self._dimension = dimension
        self._lower_bounds = lower_bounds
        self._upper_bounds = upper_bounds
        self._population_size = population_size
        self._food_size = self._population_size / 2
        self._max_trial_limit = max_trial_limit
        self._max_num_iterations = max_num_iterations
        
        self._foods = self._create_food(self._food_size)
        self._past_foods = []
        
        self._objective_values = self._calculate_objective_values(self._foods)
        self._fitness_values = self._calculate_fitness_values(self._objective_values)
        
        self._trials = np.zeros(self._food_size)
        
        self._best_index = np.argmin(self._objective_values)
        self._global_min_objective_value = self._objective_values[self._best_index]
        self._global_min_objective_values = [self._global_min_objective_value]
        self._global_best_food = self._foods[self._best_index,:]
        
        self._debug = debug
    
    def _create_food(self, size):
        food = np.random.random((size, dim)) *\
                np.tile(upper_bounds - lower_bounds, (size,1)) +\
                np.tile(lower_bounds, (size,1))
        return food
    
    def _calculate_objective_values(self, foods):
        return np.array([self._objective_function(solution) for solution in foods])
    
    def _calculate_fitness_values(self, objective_values):
        return np.array([1. / (objective_value + 1.) 
                         if objective_value >= 0. 
                         else 1. + np.abs(objective_value) 
                         for objective_value in objective_values])
    def _evaluate(self, i):
            j = np.random.random_integers(self._dimension) - 1
            k = np.random.random_integers(self._food_size) - 1
            while k == i:
                k = np.random.random_integers(self._food_size) - 1

            v_i = self._foods[i,:].copy()
            x_ij = self._foods[i,j]
            x_kj = self._foods[k,j]
            phi_ij = 2 * (np.random.rand() - 0.5)
            v_i[j] = x_ij + phi_ij * (x_ij - x_kj)

            # Check if parameter is outside bounds. If so set it to the 
            # limit.
            v_i = np.where(v_i < upper_bounds, v_i, upper_bounds).copy()
            v_i = np.where(v_i > lower_bounds, v_i, lower_bounds).copy()

            objective_value = self._calculate_objective_values([v_i])
            fitness_value = self._calculate_fitness_values(objective_value)

            if fitness_value > self._fitness_values[i]:
                self._foods[i,:] = v_i.copy()
                self._fitness_values[i] = fitness_value.copy()
                self._objective_values[i] = objective_value.copy()
                self._trials[i] = 0
            else:
                self._trials[i] = self._trials[i] + 1
        
    def _calculate_probabilities(self, fitness_values):
        '''p_i = fit_i / sum_i^n fit_i'''
        prob = fitness_values / np.sum(fitness_values)
#         prob = (0.9 * fitness_values / max(fitness_values)) + 0.1
        return prob
    
    def solve(self):
        for iteration in range(self._max_num_iterations):
            # Employed bee phase
            for i in range(self._food_size):
                self._evaluate(i)

            # Calculate probabilities. 
            # p_i = fit_i / sum_i^n fit_i
            probabilities = self._calculate_probabilities(self._fitness_values)

            # Onloooker bee phase
            i = 0
            t = 0
            while t < self._food_size:
                do_forage = np.random.rand() < probabilities[i]
                if do_forage:
                    t = t + 1
                    self._evaluate(i)
                i = i + 1
                if i == self._food_size:
                    i = 0

            # Update best solution
            best_index = np.argmin(self._objective_values)
            objective_value = self._objective_values[best_index]
            if objective_value < self._global_min_objective_value:
                self._best_index = best_index
                self._global_min_objective_value = objective_value
                self._global_best_food = self._foods[best_index,:]

            # Scout bee phase
            max_trial_idx = np.argmax(self._trials)
            max_trial = self._trials[max_trial_idx]
            if max_trial > self._max_trial_limit:
                self._trials[max_trial_idx] = 0
                v_i = self._create_food(1)
                objective_value = self._calculate_objective_values([v_i])
                fitness_value = self._calculate_fitness_values(objective_value)
                self._foods[max_trial_idx, :] = v_i.copy()
                self._fitness_values[max_trial_idx] = fitness_value.copy()
                self._objective_values[max_trial_idx] = objective_value

            self._past_foods.append(self._foods.copy())
            self._global_min_objective_values.append(self._global_min_objective_value)
            if self._debug:
                if not iteration % 50:
                    
                    print('Iteration: {}, f(x_i)_min: {}, i: {}'.format(iteration, 
                                                                        self._global_min_objective_value, 
                                                                        self._best_index))
        return self._global_best_food
    
    def plot(self):
        plt.plot(self._global_min_objective_values)
        plt.xscale('log')

In [ ]:
population_size = 20
food_size = population_size / 2
max_trial_limit = 100
max_num_iterations = 2000
obj_function = sphere
dim = 2
upper_bounds = np.ones((1,dim)) * 100
lower_bounds = np.ones((1,dim)) * -100

In [ ]:
abc = ArtificialBeeColonyAlgorithm(obj_function, dim, 
                                   upper_bounds, lower_bounds, 
                                   population_size, max_trial_limit, 
                                   max_num_iterations, debug=False)
abc.solve()

In [ ]:
abc.plot()

In [ ]:
import matplotlib.animation as animation

fig = plt.figure()
plt.clf()
colors = np.random.random((food_size, 3))
scat = plt.scatter(abc._past_foods[0][:,0], abc._past_foods[0][:,1], c=colors)


def update_plot(i, data, scat):
    data = abc._past_foods[i]
    scat.set_offsets(data)
    return scat,

ani = animation.FuncAnimation(fig,
                              update_plot, frames=xrange(max_num_iterations),
                                  fargs=([], scat),interval=100, blit=False)
plt.show()

In [ ]:
color_data.shape

In [ ]:
i = 100
scat = plt.scatter(abc._past_foods[i][:,0], abc._past_foods[i][:,1])

In [ ]:
np.array(abc._past_foods)